# Was beeinflusst die Skip Rate eines Songs?

Das gezielte Überspringen von Songs ist ein alltäglicher Bestandteil der Nutzung von Musikstreaming-Plattformen. Dabei wird ein Song manuell weitergeklickt, bevor er vollständig abgespielt wurde. Die sogenannte Skip Rate beschreibt den Anteil der Nutzer*innen, die einen bestimmten Song überspringen.

Ziel unserer Untersuchung ist es, besser zu verstehen, welche Merkmale mit der Skip Rate eines Songs in Verbindung stehen. Dazu gehören beispielsweise das Genre, die Popularität des Künstlers oder der Künstlerin, die Plattformnutzung (z. B. Free oder Premium), das Erscheinungsjahr oder die durchschnittliche Hördauer.

Als Datengrundlage nutzen wir einen Datensatz mit globalen Streamingzahlen aus dem Jahr 2024 von Spotifiy ([3]), bereitgestellt über die Plattform Kaggle. Der Datensatz enthält unter anderem Informationen zu Künstler*innen, Songs, Genres, Plattformtyp, Streams, Hörer*innenzahlen und Skip Rate.

Ziel ist es nicht nur, bestimmte Einflüsse zu identifizieren, sondern auch kritisch zu reflektieren, welche Merkmale im gegebenen Datensatz möglicherweise keine Rolle spielen und warum.

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from factor_analyzer.rotator import Rotator

In [ ]:
data_path = "data/spotify.csv"
try:
    df = pd.read_csv(data_path)
except:
    # Achtung der Pfad "../../../_Daten" muss existieren.
    import kagglehub
    import shutil
    # Download latest version
    path = kagglehub.dataset_download("atharvasoundankar/spotify-global-streaming-data-2024")
    
    # Get the first file in path
    files = os.listdir(path)
    if len(files) == 0:
        raise ValueError("No files found in the downloaded dataset.")
    # Check if the data directory exists, if not create it
    if not os.path.exists("data"):
        os.makedirs("data")

    # Move the downloaded file to the desired directory
    shutil.move(os.path.join(path, files[0]), data_path)

    print("Spotifiy dataset downloaded successfully!")
    df = pd.read_csv(data_path)

# 1. Explorative Datenanalyse

Die explorative Datenanalyse (EDA) dient dazu, einen strukturierten ersten Überblick über einen Datensatz zu gewinnen. Ziel ist es, die Verteilung der Merkmale, potenzielle Ausreißer, Zusammenhänge zwischen Variablen sowie mögliche Datenprobleme zu identifizieren, bevor weiterführende Analysen durchgeführt werden. ([1])

In diesem Zusammenhang wird die EDA verwendet, um zentrale Eigenschaften des Spotify-Datensatzes zu untersuchen. Dabei liegt der Fokus insbesondere auf Merkmalen, die potenziell mit der Skip Rate von Songs in Verbindung stehen könnten, etwa Genre, Künstler, Streaming-Plattform oder Hördauer. Zur Anwendung kommen unter anderem deskriptive Statistiken sowie visuelle Verfahren wie Histogramme, Boxplots und Streudiagramme, um Muster in den Daten sichtbar zu machen.

## 1.1 Überblick über den Datensatz

Im Folgenden wird ein erster Blick auf den Datensatz geworfen. Dabei interessieren uns insbesondere:
- die verfügbaren Spalten,
- der generelle Aufbau der Daten (z. B. Anzahl der Zeilen, Datentypen),
- sowie ein erster Eindruck der enthaltenen Informationen.

Diese erste Sichtung hilft dabei, ein Gefühl dafür zu bekommen, welche Merkmale später für die Analyse der Skip Rate von Bedeutung sein könnten.


In [ ]:
# Überblick über Struktur und Datentypen
df.info()

In [ ]:
# Vorschau auf die ersten Zeilen im Datensatz
df.head()

In [ ]:
# Alle Spaltennamen anzeigen
df.columns.tolist()

## 1.2 Deskriptive Analyse der Zielgröße: Skip Rate (%)

Zunächst wird die Verteilung der Zielvariablen `Skip Rate (%)` betrachtet. Ziel ist es, ein Gefühl für typische Werte, Ausreißer und die Streuung zu bekommen.

In [ ]:
# Deskriptive Statistik
skip_stats = df["Skip Rate (%)"].describe(percentiles=[.05, .25, .5, .75, .95])
print(skip_stats)

# Histogramm
plt.figure(figsize=(8, 5))
sns.histplot(df["Skip Rate (%)"], bins=20, kde=True)
plt.title("Verteilung der Skip Rate (%)")
plt.xlabel("Skip Rate (%)")
plt.ylabel("Häufigkeit")
plt.grid(True)
plt.show()

# Boxplot
plt.figure(figsize=(6, 2))
sns.boxplot(x=df["Skip Rate (%)"])
plt.title("Boxplot der Skip Rate (%)")
plt.xlabel("Skip Rate (%)")
plt.grid(True, axis='x')
plt.show()

Die Skip Rate in diesem Datensatz liegt im Durchschnitt bei etwa 20 %, wobei die meisten Songs Werte zwischen 10 % und 30 % aufweisen.
Einige Songs stechen jedoch mit sehr niedriger oder sehr hoher Skip Rate hervor. So liegt das untere 5. Perzentil bei lediglich 3.7 %, das obere 5. Perzentil hingegen bei über 37 %.

Insgesamt zeigt sich eine relativ gleichmäßige Verteilung ohne starke Schieflage, was darauf hinweist, dass das Überspringverhalten nicht durch eine einzelne Gruppe dominiert wird.
Die Werte deuten darauf hin, dass sowohl besonders selten übersprungende als auch besonders häufog übersprungende Songs existieren, die in weiteren Analysen gezielt betrachtet werden könnten.

## 1.3 Skip Rate nach Musikgenre

Da Musikgeschmack stark genreabhängig ist, könnte auch die Skip Rate mit dem Genre eines Songs zusammenhängen.
Unterschiedliche Genres sprechen unterschiedliche Zielgruppen an und können sich hinsichtlich Aufbau, Länge und Hörgewohnheiten stark unterscheiden.

Im Folgenden wird untersucht, wie hoch die durchschnittliche Skip Rate je Genre ausfällt.

In [ ]:
# Durchschnittliche Skip Rate pro Genre berechnen
genre_means = df.groupby("Genre")["Skip Rate (%)"].mean().sort_values(ascending=False)

# Plot erstellen
plt.figure(figsize=(10, 8))
sns.barplot(
    x=genre_means.values,
    y=genre_means.index,
    hue=genre_means.index,
    palette="viridis",
    dodge=False,
    legend=False
)
plt.xlabel("Durchschnittliche Skip Rate (%)")
plt.ylabel("Genre")
plt.title("Durchschnittliche Skip Rate nach Musikgenre")
plt.tight_layout()
plt.show()

Die Analyse zeigt klare Unterschiede in der durchschnittlichen Skip Rate je Musikgenre. Besonders auffällig ist hier, dass Pop-Songs am häufigsten übersprungen werden, gefolgt von Hip Hop, Rock und EDM. Genres wie Indie, K-pop und klassische Musik weisen dagegen die niedrigsten Skip Rates im Datensatz auf.

Ein möglicher Erklärungsansatz ist, dass populäre Genres wie Pop häufiger konsumiert werden. Nutzer*innen skippen eher, wenn ihnen der Song nicht sofort gefällt. Weniger massentaugliche Genres wie Indie oder Classical könnten dagegen gezielter ausgewählt und daher seltener übersprungen werden.

Diese Beobachtungen legen nahe, dass das Genre tatsächlich ein relevanter Prädiktor für die Skip Rate sein könnte und in weiteren Analysen berücksichtigt werden sollte.

## 1.4 Streuung der Skip Rate innerhalb der Genres

Nachdem die durchschnittliche Skip Rate je Genre betrachtet wurde, wird nun die Verteilung innerhalb der einzelnen Genres analysiert.
Boxplots zeigen dabei sowohl typische Werte (Median), als auch die Streuung (Interquartilsabstand) und eventuelle Ausreißer.

In [ ]:
# Boxplot der Skip Rate pro Genre
plt.figure(figsize=(12, 8))
sns.boxplot(data=df, x="Skip Rate (%)", y="Genre", hue="Genre", palette="viridis", legend=False)
plt.title("Verteilung der Skip Rate innerhalb der Genres")
plt.xlabel("Skip Rate (%)")
plt.ylabel("Genre")
plt.tight_layout()
plt.show()

Die Boxplots zeigen, dass die Skip Rate innerhalb der meisten Genres stark variiert. Für nahezu alle Genres finden sich sowohl Songs mit extrem niedriger als auch sehr hoher Skip Rate. Das legt nahe, dass das Genre zwar ein relevanter Faktor für die Skip Rate im Mittel ist, aber nicht allein entscheidend.

## 1.5 Zusammenhang zwischen Skip Rate und monatlicher Hörerzahl

Ein möglicher Einflussfaktor auf die Skip Rate ist die Popularität der Künstler*innen.
Es liegt nahe anzunehmen, dass Songs von sehr bekannten Künstler**innen mit vielen monatlichen Hörer*innen seltener übersprungen werden, z. B. weil sie bekannter oder erwarteter sind.

Im Folgenden wird dieser Zusammenhang mittels eines Scatterplots dargestellt.

In [ ]:
# Scatterplot: Skip Rate vs. Monthly Listeners
plt.figure(figsize=(8, 6))
sns.regplot(
    data=df,
    x="Monthly Listeners (Millions)",
    y="Skip Rate (%)",
    scatter_kws={'alpha':0.4, 's':30},
    line_kws={'color': 'red'}
)
plt.title("Skip Rate vs. Monthly Listeners (mit Trendlinie)")
plt.xlabel("Monatliche Hörer*innen (in Millionen)")
plt.ylabel("Skip Rate (%)")
plt.grid(True)
plt.tight_layout()
plt.show()


# Korrelationskoeffizient berechnen
corr = df["Monthly Listeners (Millions)"].corr(df["Skip Rate (%)"])
print(f"Korrelation zwischen Skip Rate und Monthly Listeners: {corr:.2f}")

Die Analyse zeigt, dass es keinen erkennbaren Zusammenhang zwischen der Skip Rate eines Songs und der monatlichen Hörerzahl der Künstler*innen gibt. Dies bestätigt auch der Korrelationswert von 0.01 – also praktisch null.

Daraus lässt sich schließen, dass die Popularität eines Künstlers kein verlässlicher Prädiktor dafür ist, wie häufig ein Song übersprungen wird. Sowohl sehr beliebte als auch wenig bekannte Künstler*innen können Songs mit hoher oder niedriger Skip Rate haben.

## 1.6 Skip Rate in Abhängigkeit vom Erscheinungsjahr

Ein weiterer potenzieller Einflussfaktor auf das Überspringverhalten könnte das Erscheinungsjahr eines Songs sein.
Es ist denkbar, dass Songs, die vor kurzem veröffentlicht wurden, entweder besonders oft gehört oder auch schneller wieder übersprungen werden, z. B. aufgrund von Trends.

In [ ]:
# Durchschnittliche Skip Rate pro Jahr
year_means = df.groupby("Release Year")["Skip Rate (%)"].mean()

# Plot
plt.figure(figsize=(10, 6))
sns.lineplot(x=year_means.index, y=year_means.values, marker="o")
plt.title("Durchschnittliche Skip Rate pro Erscheinungsjahr")
plt.xlabel("Erscheinungsjahr")
plt.ylabel("Skip Rate (%)")
plt.grid(True)
plt.tight_layout()
plt.show()

Zwischen 2018 und 2023 zeigt sich ein leichter, aber auffälliger Trend: Die durchschnittliche Skip Rate sinkt zunächst bis zum Jahr 2021 deutlich ab, bevor sie in den Jahren 2022 und 2023 wieder leicht ansteigt. Besonders auffällig ist das Jahr 2021, das mit einer durchschnittlichen Skip Rate von nur ca. 16 % einen deutlichen Ausreißer nach unten darstellt.
Insgesamt lässt sich jedoch kein klarer linearer Zusammenhang zwischen Erscheinungsjahr und Skip Rate feststellen.

## 1.7 Skip Rate nach Plattformtyp

Die Art der genutzten Plattform, etwa ob ein Nutzer oder eine Nutzerin ein kostenloses oder kostenpflichtiges Abo verwendet, kann das Hörverhalten beeinflussen.

In [ ]:
# Mittelwert der Skip Rate je Plattformtyp berechnen
platform_means = df.groupby("Platform Type")["Skip Rate (%)"].mean().sort_values()

# Plot
plt.figure(figsize=(8, 5))
sns.barplot(
    x=platform_means.values,
    y=platform_means.index,
    hue=platform_means.index,
    palette="viridis",
    dodge=False,
    legend=False
)
plt.xlabel("Durchschnittliche Skip Rate (%)")
plt.ylabel("Plattformtyp")
plt.title("Skip Rate nach Plattformtyp")
plt.tight_layout()
plt.show()

for platform, skiprate in platform_means.items():
    print(f"{platform}: {skiprate:.2f} %")

Die Analyse zeigt, dass der Plattformtyp keinen signifikanten Unterschied in der durchschnittlichen Skip Rate aufweist. Beide Gruppen liegen mit einer durchschnittlichen Skip Rate von etwas über 20 % nahezu gleichauf.

## 1.8 Skip Rate in Abhängigkeit von der durchschnittlichen Stream-Dauer

Ein möglicher Einflussfaktor auf das Überspringverhalten ist die durchschnittliche Hördauer eines Songs. Die Variable `Avg Stream Duration (Min)` zeigt, wie lange ein Song im Durchschnitt pro Stream gehört wird.

In [ ]:
# Scatterplot: Skip Rate vs. durchschnittliche Hördauer
plt.figure(figsize=(8, 6))
sns.regplot(
    data=df,
    x="Avg Stream Duration (Min)",
    y="Skip Rate (%)",
    scatter_kws={'alpha':0.4, 's':30},
    line_kws={'color': 'red'}
)
plt.title("Skip Rate vs. durchschnittliche Stream-Dauer")
plt.xlabel("Durchschnittliche Stream-Dauer (Minuten)")
plt.ylabel("Skip Rate (%)")
plt.grid(True)
plt.tight_layout()
plt.show()

# Korrelationskoeffizient berechnen
corr_duration = df["Avg Stream Duration (Min)"].corr(df["Skip Rate (%)"])
print(f"Korrelation zwischen Skip Rate und Stream-Dauer: {corr_duration:.2f}")

Es zeigt, dass zwischen der durchschnittlichen Hördauer eines Songs und dessen Skip Rate kein nennenswerter Zusammenhang besteht. Der Korrelationswert von –0.02 bestätigt, dass sich die beiden Variablen praktisch unabhängig voneinander verhalten.

## 1.9 Fazit der explorativen Datenanalyse

Im Rahmen dieser explorativen Datenanalyse wurde untersucht, welche Faktoren potenziell mit der Skip Rate von Songs auf Spotify zusammenhängen. Ziel war es, auf Basis des Datensatzes "Spotify Global Streaming Data 2024" Hypothesen zu formulieren und erste Zusammenhänge sichtbar zu machen.

Die wichtigsten Erkenntnisse im Überblick:

- **Genre**: Der deutlichste Einflussfaktor. Es zeigten sich klare Unterschiede in der durchschnittlichen Skip Rate zwischen verschiedenen Musikrichtungen. Pop-Songs wiesen die höchste durchschnittliche Skip Rate auf, während z. B. Indie, K-pop und klassische Musik eher seltener übersprungen wurden. Auch die Streuung innerhalb der Genres war teils sehr unterschiedlich.

- **Plattformtyp (Free vs. Premium)**: Es konnte kein relevanter Unterschied in der durchschnittlichen Skip Rate festgestellt werden. Das Überspringverhalten scheint unabhängig vom Abo-Modell zu sein.

- **Beliebtheit (Monthly Listeners)**: Der Korrelationswert von nur 0.01 deutet auf keinen Zusammenhang zwischen der Popularität eines Künstlers und der Skip Rate seiner Songs hin.

- **Erscheinungsjahr**: Ein leichter Trend mit einem auffälligen Rückgang der Skip Rate im Jahr 2021 war erkennbar, jedoch ohne klaren linearen Zusammenhang. Aufgrund der begrenzten Jahrgangsbreite im Datensatz ist diese Analyse nur eingeschränkt aussagekräftig.

- **Durchschnittliche Stream-Dauer**: Auch hier zeigte sich kein relevanter Zusammenhang. Die Skip Rate war weitgehend unabhängig davon, wie lange ein Song im Schnitt pro Stream gehört wurde.

### Zusammenfassung:

Die explorative Datenanalyse zeigt, dass insbesondere das Genre eines Songs ein zentraler Prädiktor für das Überspringverhalten sein kann. Andere Merkmale wie Beliebtheit, Plattform oder Hördauer haben dagegen keinen erkennbaren Einfluss im gegebenen Datensatz.

# 2 Multifaktorielle Regression
Nachdem im ersten Kapitel die wichtigsten Einflussfaktoren auf die Skip Rate von Songs explorativ untersucht wurden, folgt nun eine vertiefende Analyse mittels multifaktorieller Regressionsverfahren ([2]). Ziel dieses Kapitels ist es, die gleichzeitigen Effekte mehrerer Variablen auf die Skip Rate quantitativ zu bestimmen und deren relative Bedeutung zu bewerten.

Dazu werden zunächst alle relevanten Merkmale des Datensatzes – darunter Genre, Plattformtyp, Popularität, Erscheinungsjahr und weitere – in ein Regressionsmodell integriert. Durch die Berücksichtigung mehrerer Einflussgrößen gleichzeitig kann überprüft werden, welche Faktoren auch unter Kontrolle anderer Variablen einen signifikanten Beitrag zur Erklärung der Skip Rate leisten.

Die für die Regression benötigten kategorialen Variablen wie Genre, Plattformtyp, Land, Künstler*in und Album werden vor der Modellierung in numerische Variablen umgewandelt. Dies geschieht mithilfe des sogenannten One-Hot-Encoding: Für jede Ausprägung einer Kategorie wird eine eigene Spalte erzeugt, die den Wert 1 annimmt, wenn der jeweilige Song diese Ausprägung besitzt, und sonst 0. Dadurch können auch nicht-numerische Merkmale in das Regressionsmodell integriert werden. Um Redundanzen zu vermeiden, wird jeweils die erste Kategorie einer Variable ausgelassen (drop_first=True). So entsteht eine rein numerische Datenmatrix, die für die Regressionsanalyse geeignet ist.

In [ ]:
df_encoded = pd.get_dummies(df, columns=['Country', 'Genre', 'Platform Type', 'Artist', 'Album'], drop_first=True)

## 2.1 Korrelation
Die Korrelationsmatrix wird genutzt, um die Zusammenhänge zwischen Variablen im Datensatz zu untersuchen. Sie zeigt, wie stark und in welche Richtung einzelne Merkmale miteinander korrelieren. Besonders interessant ist dabei die Korrelation der einzelnen Features mit der Zielvariable „Skip Rate (%)“, um potenziell relevante Einflussfaktoren für die weitere Modellierung zu identifizieren.

In [ ]:
corr_matrix = df_encoded.corr(numeric_only=True)
plt.figure(figsize=(12,8))
sns.heatmap(corr_matrix, cmap='coolwarm', center=0)
plt.title("Korrelationsmatrix (df_encoded)")
plt.show()

Die erhaltene Korrelationsmatrix zeigt die paarweisen Zusammenhänge zwischen allen Variablen des kodierten Datensatzes. In der Matrix sind die Werte zwischen -1 und 1 angegeben: Ein Wert nahe 1 bedeutet eine starke positive Korrelation, ein Wert nahe -1 eine starke negative Korrelation, und Werte um 0 deuten auf keinen linearen Zusammenhang hin. Die Visualisierung als Heatmap erleichtert es, Muster und auffällige Korrelationen auf einen Blick zu erfassen. So ist zu sehen das eine extreme Korrelation zwischen Artist und den Alben die der jeweilige Artist geschrieben hat existiert.

Die Skip Rate zeigt in der Korrelationsmatrix nur geringe Korrelationen mit den meisten anderen Variablen, sodass sich keine klaren linearen Zusammenhänge ablesen lassen. Um dennoch einen Eindruck von der Stärke und Verteilung der Korrelationen zu erhalten, wird ergänzend ein Boxplot der Korrelationskoeffizienten dargestellt. Dieser visualisiert, wie schwach oder stark die einzelnen Merkmale insgesamt mit der Skip Rate zusammenhängen.

In [ ]:
plt.figure(figsize=(6, 2))
sns.boxplot(x=corr_matrix['Skip Rate (%)'].drop('Skip Rate (%)'))
plt.title("Boxplot der Korrelationen mit Skip Rate (%)")
plt.xlabel("Korrelationskoeffizient")
plt.grid(True, axis='x')
plt.show()

In [ ]:
corr_target = corr_matrix['Skip Rate (%)'].sort_values(ascending=False)
print("Top 5 höchste Korrelationen mit Skip Rate (%):")
print(corr_target.head(5))
print("\nTop 5 niedrigste Korrelationen mit Skip Rate (%):")
print(corr_target.tail(5))

Um die Relevanz einzelner Merkmale für die Skip Rate besser einschätzen zu können, werden zusätzlich die Extremwerte der Korrelationskoeffizienten ausgegeben. Dadurch lässt sich erkennen, welche Variablen die höchsten positiven bzw. negativen Zusammenhänge mit der Skip Rate aufweisen, auch wenn diese insgesamt eher schwach ausfallen.

## 2.2 Multifaktorielle Regression
Für die Regressionsanalyse werden die Daten in unabhängige Variablen (Features) und die abhängige Variable (Skip Rate) aufgeteilt. Anschließend wird der Datensatz in Trainings- und Testdaten unterteilt, wobei 20 % der Daten als Testmenge verwendet werden.

In [ ]:
X = df_encoded.drop(columns=['Skip Rate (%)'])
y = df_encoded['Skip Rate (%)']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print("R^2:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))


X_numeric = X.apply(pd.to_numeric, errors='coerce')
X_numeric = X_numeric.astype(float)
X_sm = sm.add_constant(X_numeric)
model_sm = sm.OLS(y, X_sm).fit()
pvalues = model_sm.pvalues[X.columns]

coef_df = pd.DataFrame({
    "Feature": X.columns,
    "Koeffizient": model.coef_,
    "p-Wert": pvalues
}).sort_values(by="Koeffizient", key=abs, ascending=False)
print(coef_df.head(10))

R² beträgt -0,12. Dieser negativer R²-Wert bedeutet, dass das Modell schlechter vorhersagt als der Mittelwert der Zielvariable. Das Modell kann die Skip Rate also nicht sinnvoll erklären.

Der mittlere quadratische Fehler (MSE) liegt bei 128,34. Das ist ein Maß für die durchschnittliche Abweichung der Vorhersagen von den tatsächlichen Werten – je kleiner, desto besser. Hier ist der Wert relativ hoch.

Die Tabelle zeigt die zehn Merkmale mit den größten absoluten Regressionskoeffizienten sowie deren p-Werte. Ein negativer Koeffizient bedeutet, dass das jeweilige Merkmal die Skip Rate senkt, während ein positiver Wert auf eine Erhöhung der Skip Rate hinweist. Zu den wichtigsten Einflussfaktoren zählen vor allem Länder wie „Country_Germany“ und „Country_United Kingdom“ sowie verschiedene Genres, beispielsweise „Genre_Pop“ oder „Genre_Hip Hop“. Die p-Werte geben an, wie statistisch signifikant die jeweiligen Koeffizienten sind. Von den aufgeführten Merkmalen ist lediglich „Country_Germany“ mit einem p-Wert von 0,049 knapp signifikant, während die übrigen Merkmale keine statistische Signifikanz aufweisen.

In [ ]:
feat1 = "Monthly Listeners (Millions)"
feat2 = "Total Streams (Millions)"

xs = df[feat1]
ys = df[feat2]
zs = df["Skip Rate (%)"]

fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs, ys, zs, c=zs, cmap='viridis', alpha=0.7)

X_plane = df[[feat1, feat2]]
y_plane = df["Skip Rate (%)"]
model3d = LinearRegression().fit(X_plane, y_plane)

x_surf, y_surf = np.meshgrid(
    np.linspace(xs.min(), xs.max(), 20),
    np.linspace(ys.min(), ys.max(), 20)
)
z_surf = model3d.intercept_ + model3d.coef_[0]*x_surf + model3d.coef_[1]*y_surf

ax.plot_surface(x_surf, y_surf, z_surf, alpha=0.3, color='orange')

ax.set_xlabel(feat1)
ax.set_ylabel(feat2)
ax.set_zlabel('Skip Rate (%)')
plt.title("3D-Regression: Skip Rate (%)")
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel("Wahre Skip Rate (%)")
plt.ylabel("Vorhergesagte Skip Rate (%)")
plt.title("Regression: Wahre vs. vorhergesagte Skip Rate")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.show()

Der Scatterplot stellt die tatsächlichen Skip-Rate-Werte den vom Regressionsmodell vorhergesagten Werten gegenüber. Jeder Punkt im Diagramm entspricht dabei einer Zeile aus den Testdaten: Die x-Achse zeigt die reale Skip Rate, die y-Achse die vom Modell berechnete Skip Rate. Idealerweise würden alle Punkte auf einer Diagonalen liegen, was bedeuten würde, dass das Modell die Skip Rate exakt vorhersagen kann.

Gerade deshalb ist der Scatterplot so relevant: Er macht auf einen Blick sichtbar, wie schlecht das Modell arbeitet. Die Punkte sind weit gestreut und folgen keiner klaren Linie. Das zeigt, dass das Modell die tatsächlichen Werte nicht zuverlässig abbilden kann. Der Scatterplot ist somit ein zentrales Werkzeug, um die Vorhersagekraft eines Regressionsmodells zu beurteilen und Schwächen unmittelbar zu erkennen.

Wie sowohl das 3D-Modell als auch der Scatterplot deutlich zeigen, ist das verwendete Regressionsmodell nicht geeignet, die Skip Rate zuverlässig vorherzusagen. Im 3D-Modell, das die Skip Rate in Abhängigkeit von zwei ausgewählten Merkmalen darstellt, ist keine klare Struktur oder ein Zusammenhang zwischen den Variablen und der Zielgröße erkennbar. Die Punkte verteilen sich weitgehend ungeordnet im Raum, und die Regressionsfläche passt sich den Daten kaum sinnvoll an. Diese Beobachtungen spiegeln sich auch in den schlechten Gütemaßen wider und verdeutlichen, dass das Modell für die Vorhersage der Skip Rate ungeeignet ist.

## 2.3 Fazit
Die multifaktorielle Regression zeigt, dass sich die Skip Rate von Alben mit den eingesetzten multifaktoriellen Regressionsverfahren nicht zuverlässig vorhersagen lässt. Weder das lineare Regressionsmodell noch die Analyse der Regressionskoeffizienten liefern überzeugende Ergebnisse: Die Gütemaße wie der negative R²-Wert und der hohe mittlere quadratische Fehler deuten darauf hin, dass das Modell die tatsächlichen Werte schlechter abbildet als eine einfache Mittelwertschätzung. Auch die Visualisierungen, insbesondere das 3D-Modell und der Scatterplot, machen deutlich, dass keine klaren Zusammenhänge zwischen den untersuchten Merkmalen und der Skip Rate bestehen. Die Punkte sind weit gestreut und zeigen keine erkennbare Struktur. Insgesamt lässt sich festhalten, dass die betrachteten Merkmale im gegebenen Datensatz nicht ausreichen, um die Skip Rate mit klassischen Regressionsmethoden sinnvoll zu modellieren.



# 3. Hauptkomponenten Analyse
Es wurde gezeigt, dass eine Multifaktorielle Regression die Skip Rate nicht vorhersagen kann. Daher wird im folgenden
eine Hauptkomponenten Analyse durchgeführt um mit den gefundenen Komponenten eine besseres Modell zu bilden.

Die Hauptkomponentenanalyse (PCA) ist ein Verfahren zur Dimensionsreduktion, das darauf abzielt, die wesentlichen Strukturen in hochdimensionalen Datensätzen zu erkennen. Sie transformiert die ursprünglichen Variablen in eine neue Menge unkorrelierter Variablen, die sogenannten Hauptkomponenten. Diese Hauptkomponenten sind so konstruiert, dass die erste Komponente die größte mögliche Varianz der Daten erklärt, die zweite die nächstgrößte und so weiter. Die Transformation erfolgt durch eine lineare Kombination der Ausgangsvariablen, wobei die Gewichte (Ladungen) die Bedeutung der einzelnen Variablen für die jeweilige Komponente angeben. 

PCA ermöglicht es, Muster und Zusammenhänge in den Daten zu erkennen, die mit bloßem Auge schwer zu identifizieren wären. Die Interpretation der Hauptkomponenten erfolgt über die Ladungen: Hohe positive oder negative Werte zeigen, welche Variablen besonders stark zur jeweiligen Komponente beitragen. Durch die Reduktion auf wenige Hauptkomponenten kann man die Komplexität der Daten verringern und gleichzeitig einen Großteil der Information bewahren. ([1])

## 3.1 Datenvorbereitung

Zunächst werden die kategorialen Variablen des Datensatzes mit einem One-Hot-Encoding umgeformt.

In [ ]:
# Teilung der Test Daten zur Vorhersage der Skip Rate (%) mit Kodierung kategorialer Variablen
# Kategoriale Variablen werden automatisch per One-Hot-Encoding kodiert.
# Feature- und Zielspalten
feature_cols = [
    'Country', 
    'Artist', 
    'Album', 
    'Genre', 
    'Release Year',
    'Monthly Listeners (Millions)', 
    'Total Streams (Millions)',
    'Total Hours Streamed (Millions)', 
    'Avg Stream Duration (Min)',
    'Streams Last 30 Days (Millions)'
]

target_col = 'Skip Rate (%)'

# Filtere und bereite die Daten vor
df_reg = df[feature_cols + [target_col]].dropna()
X = df_reg[feature_cols]
y = df_reg[target_col]

# Kategoriale Variablen kodieren mit One-Hot-Encoding
X_encoded = pd.get_dummies(X, drop_first=True)

Bevor eine Hauptkomponentenanalyse durchgeführt wird, werden alle Variablen Standertisiert um Vergleichbarkeit zwischen den Variablen herzustellen.

In [ ]:
# Hauptkomponentenanalyse (PCA) auf den kodierten Features
# Die Features werden vor der PCA standardisiert.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)


## 3.2 Komponentenselektion

Zur Selektion der Hauptkomponenten suchen wir zunächst, wie viele Komponenten es brauch um 90% der Varianz der Daten zu erklären. 
Die gefundenen Komponenten stellen wir nun mit ihrem Eigenwert in einem Scree-Plot dar. An diesem kann ein signifikanter Sprung identifiziert werden, um zu Entscheiden ab welchen Komponentenanzahl der Cutoff gewählt wird.
Visuell auffällig sind die Sprunge zur Zweiten, Zwölften und Dreizehnten Komponente. Alle Komponente zwischen von dem Ersten bis zur 13. haben einen Eigenwert größer als eins und genügen damit dem Kaiser-Kriterium. 
Da eine weitere Analyse mit nur einem Kriterium reduntant wäre, wird sich für den nächsten signifikanten Knick an der Komponente zwölf entschieden. 

In [ ]:
# Führe PCA durch, sodass mindestens 90% der Varianz erklärt werden
pca = PCA(n_components=0.9, svd_solver='full')
X_pca = pca.fit_transform(X_scaled)

# Scree-Plot: Eigenwerte der Hauptkomponenten
plt.figure(figsize=(8,4))
plt.plot(range(1, len(pca.explained_variance_)+1), pca.explained_variance_, marker='o')
plt.axhline(1, color='red', linestyle='--', label='Grenze = 1')
plt.xlabel('Hauptkomponente')
plt.ylabel('Eigenwert')
plt.title('Scree-Plot der Hauptkomponenten (PCA)')
plt.legend()
plt.tight_layout()
plt.show()


Die zwölf selektieren Komponeten erkären gemeinsam 45,66% der Varianz der Daten 

In [ ]:
pca = PCA(n_components=12, svd_solver='full')
X_pca = pca.fit_transform(X_scaled)

print(f"Erklärte Varianz: {np.sum(pca.explained_variance_ratio_)*100:.2f}%")


## 3.3 Komponentenidentifikation

Nun stellen wir die Ladungen der Variablen auf den Komponenten dar. Dazu wird eine Varimax transformation genutzt um starke Ladungen hervorzuheben und schwache Ladungen zu unterdrücken. ([1])

An der Resultierenden Übersicht ist auffällig das jede Komponente von einem Album und ihrem jeweilgen Künstler*innen dominiert wird. 
Die allgemeinerien Variablen, wie das Erscheinungsjahr oder die Monatlichen Höhrer laden tendenziel auf alle Komponenten gleichermaßen. 
An der Darstellung ist zu beachten, dass alle Werte die unter den Wert von 0,2 fallen ausgeblendet wurden und die Tabelle lesbar zu gestalten. 

In [ ]:
# Varimax-Rotation auf die PCA-Komponentenmatrix anwenden
rotator = Rotator(method='varimax')
rotated_components = rotator.fit_transform(pca.components_.T).T

# Rotierte Komponentenmatrix als DataFrame anzeigen
rotated_df = pd.DataFrame(rotated_components, columns=X_encoded.columns, index=[f'PC{i+1}' for i in range(pca.n_components_)]).T

# Für jede Variable: Komponente mit höchster Ladung (nach Betrag) bestimmen
max_comp = rotated_df.abs().idxmax(axis=1)
max_val = rotated_df.abs().max(axis=1)
comp_num = max_comp.str.extract(r'(\d+)').astype(int)[0]

# Sortierindex erstellen: erst nach Komponente, dann nach Höhe der Ladung (absteigend)
sort_idx = pd.DataFrame({'comp': comp_num, 'val': max_val})
sort_idx = sort_idx.sort_values(['comp', 'val'], ascending=[True, False]).index
rotated_df_sorted = rotated_df.loc[sort_idx]

# Werte < 0.08 ausblenden
rotated_df_masked = rotated_df_sorted.map(lambda x: x if abs(x) >= 0.2 else "")

# Trick um das gesamte DataFrame anzuzeigen
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(rotated_df_masked)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

## 3.4 Multifaktorielle Regression mit Komponenten
Nun wird eine neue Multifaktorielle Regression trainiert, jedoch mit den gefunden Komponenten anstelle der Rohdaten. 

Im direkten Vergleich ist eine Verbesserung der Kennwerte zu sehen. Jedoch nicht in einem Maße, dass das neue Modell bessere als eine Mittelswertschätzung des Skip Rate ist. 

In [ ]:
# Multifaktorielle Regression mit den extrahierten Hauptkomponenten
# Die Zielvariable y und die extrahierten Hauptkomponenten X_pca sind bereits definiert
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.2, random_state=42)

reg_pca = LinearRegression()
reg_pca.fit(X_train_pca, y_train_pca)
y_pred_pca = reg_pca.predict(X_test_pca)

print("R² (PCA-Komponenten):", r2_score(y_test_pca, y_pred_pca))
print("MSE (PCA-Komponenten):", mean_squared_error(y_test_pca, y_pred_pca))

# 4. Entscheidungsbaum
Da eine Modellierung durch eine Multifaktorielle Regression mit Komponeten einer Hauptkomponentenanalyse nicht passend für die Schätzung der Skip Rate wird Abbschließend der Versuch unternommen ein Regressionsentscheidungsbaum trainiert.

Ein Regressionsentscheidungsbaum ist ein Modell aus dem Bereich des maschinellen Lernens, das zur Vorhersage kontinuierlicher Zielgrößen eingesetzt wird. Der Baum besteht aus Knoten, an denen die Daten anhand von Entscheidungsregeln in Teilmengen aufgeteilt werden. Jeder Endknoten des Baums gibt eine Vorhersage für die Zielgröße ab, typischerweise den Mittelwert der Zielwerte in diesem Blatt. Die Aufteilung erfolgt so, dass die Homogenität innerhalb der Blätter maximiert wird, d.h. die Varianz der Zielgröße innerhalb eines Blatts möglichst gering ist.

Auch für dieses Modell wird der in 20% Testdaten und 80% Trainingsdaten geteilt.

In [ ]:
# Splitte die Daten in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

## 4.1 Gridsuche

Da die optimalen Parameter für das Training eines Entscheidungsbaums mit den gegeben Daten unbekannt ist, wird mit einer Gridsuche systematisch Parameter ausprobiert. Es werden jene Parameter ausgewählt, welche in dem größten Wert des Bestimmtheitsmaßes, R², resultieren.

In [ ]:
# Grid Search zur Optimierung der Entscheidungsbaum-Parameter
param_grid = {
    'max_depth': [4, 6, 8, 12, None],
    'min_impurity_decrease': [0.0, 0.01, 0.1, 0.5],
    'min_samples_split': [2, 5, 10, 20]
}

grid_search = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

print("Beste Parameterkombination:")
print(grid_search.best_params_)
print(f"Bester mittlerer R²-Score: {grid_search.best_score_:.3f}")

# Optional: Ergebnisse als DataFrame anzeigen
# results_df = pd.DataFrame(grid_search.cv_results_)
# display(results_df[['params', 'mean_test_score', 'std_test_score']].sort_values('mean_test_score', ascending=False).head(10))

## 4.2 Kreuzvalidierung

Um ein Overfitting des Modells zu kontrollieren wird eine Kreuzvalidierung über 5 Holdouts berechnet. Das Ergebnis weißt eine akzeptierbare Streuung auf in Anbetracht der geringen Datenmenge.

In [ ]:
reg_tree = DecisionTreeRegressor(**grid_search.best_params_, random_state=42)

# Kreuzvalidierung für den Regressionsentscheidungsbaum
splits = 5
cv = KFold(n_splits=splits, shuffle=True, random_state=42)

# R²-Scores
r2_scores = cross_val_score(reg_tree, X_encoded, y, cv=cv, scoring='r2')
print(f"Mittlerer R²-Score ({splits}-fold CV): {np.mean(r2_scores):.3f} ± {np.std(r2_scores):.3f}")

# MSE-Scores (negativ, daher Vorzeichen umdrehen)
mse_scores = cross_val_score(reg_tree, X_encoded, y, cv=cv, scoring=make_scorer(mean_squared_error, greater_is_better=False))
print(f"Mittlerer MSE ({splits}-fold CV): {(-np.mean(mse_scores)):.3f} ± {np.std(mse_scores):.3f}")

Final wird ein Regressionsbaum trainiert dessen Bestimmtheitsmaß negativ ist. Dies lässt darauf schließen, dass das Modell schlechter auf die Daten passt als eine Mittlewertsschätzung der Skip Rate. Abschließend ist daher zu folgern, dass eine Vorhersage der Skip Rate mittels eines Regressionentscheidungsbaum nicht möglich ist. 

In [ ]:
# Vorhersage und Bewertung
reg_tree.fit(X_train, y_train)
y_pred = reg_tree.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse:.3f}, R²: {r2:.3f}")


# 5. Zusammenfassung
In dieser Arbeit wurde mit mehreren statistischen Verfahren, namentlich multifaktorieller Regression, Hauptkomponentenanalyse und Entscheidungsbaum, versucht, einen Zusammenhang zwischen der Skip Rate von Alben und ihren Eigenschaften herzustellen, wie zum Beispiel wie oft die Alben gehört wurden oder zu welchem Genre sie gehören. Jedoch konnte keines der angeführten Modelle die Skip Rate zuverlässig vorhersagen.

Es ist daher davon auszugehen, dass die Skip Rate eines Albums nicht von allgemeinen Faktoren der Alben abhängt, sondern zum Beispiel mit individuellen Faktoren der Nutzer zusammenhängt. Zum Beispiel das Land, in dem Nutzer ein Album hören, da die Regressionskoeffizienten der Länder innerhalb der multifaktoriellen Regression tendenziell die höchsten Ladungen aufwiesen.  

# 6. Weitere Forschung

Weiterführende Forschungsansätze könnten davon profitieren, Daten auf einer detailreicheren Ebene zu verwenden. Zum Beispiel könnte die Skip Rate zwischen mehreren Songs desselben Albums variieren oder jeder Nutzer ein eigenes Überspringverhalten haben. Daten, die Analysen auf solchen Ebenen ermöglichen, haben das Potenzial, aufschlussreiche Ergebnisse zu erzielen.  

Diese Arbeit versuchte einen explorativen "Rundumschlag" durchzuführen, welcher nicht geglückt ist, jedoch wertvolle Erfahrungen mit der Verwendung von statistischen Verfahren bot. Daher wäre eine Prüfung spezifischerer Fragestellungen ratsam. So zum Beispiel werden in gewissen Ländern gewisse Genres von Musik präferiert.

Abschließend könnte die selbe Forschungsfrage neu bearbeitet werden mit einer selektiveren Wahl von Variablen und Eliminierung Redundaten Information, wie zum Beispiel Künstler und ihre Alben, solange im Datensatz nicht mehrere Alben des selben Künstlers existieren.  

# Literaturverzeichnis

[1] W. A. Hemmerich, „Einstieg in die Hauptkomponentenanalyse | StatistikGuru.de“. Zugegriffen: 5. Juni 2025. [Online]. Verfügbar unter: https://statistikguru.de/spss/hauptkomponentenanalyse/einstieg-2.html

[2] B. E. Schlegel, „Multiple Lineare Regression | springerprofessional.de“, in Sozialwissenschaftliche Datenanalyse, Springer Fachmedien Wiesbaden, 2025. Zugegriffen: 5. Juni 2025. [Online]. Verfügbar unter: https://link.springer.com/chapter/10.1007/978-3-658-47201-6_11

[3] „Spotify – Webplayer: Musik für alle“, Spotify. Zugegriffen: 5. Juni 2025. [Online]. Verfügbar unter: https://open.spotify.com/intl-de
